## This file contains some potentially useful integrators/dynamics code

In [ ]:
SolarSystemParameters=dict()

SolarSystemParameters["EarthRadius"]=6.371e6 #meters
SolarSystemParameters["EarthRotatingVelocity"]=444.444 # meters per second
SolarSystemParameters["EarthOrbitRadius"]=149.6e9 # meters
SolarSystemParameters["EarthOrbitVelocity"]=29.8e3 # meters per second
SolarSystemParameters["EarthMass"]=5.972e24 # kg
SolarSystemParameters["MoonOrbitRadius"]=363228.9e3
SolarSystemParameters["MoonMass"]=7.34e22 # kg

SolarSystemParameters["SunRadius"]=695.51e3 #meters
SolarSystemParameters["SunMass"]=1.989e30 # kg
SolarSystemParameters["G"]=6.67408e-11

## Mercury
SolarSystemParameters["MercuryRadius"]=2.4397e6 #meters
SolarSystemParameters["MercuryMass"]=3.3011e23 # kg
SolarSystemParameters["MercuryOrbitRadius"]=57.91e9 # meters
SolarSystemParameters["MercuryOrbitVelocity"]=47.36e3 # meters per second

## Venus
SolarSystemParameters["VenusRadius"]=6.0518e6 #meters
SolarSystemParameters["VenusMass"]=4.8675e24 # kg
SolarSystemParameters["VenusOrbitRadius"]=108.21e9 # meters
SolarSystemParameters["VenusOrbitVelocity"]=35.02e3 # meters per second

## Mars
SolarSystemParameters["MarsRadius"]=3.3895e6 #meters
SolarSystemParameters["MarsMass"]=6.4171e23 # kg
SolarSystemParameters["MarsOrbitRadius"]=227.92e9 # meters
SolarSystemParameters["MarsOrbitVelocity"]=24.077e3 # meters per second

## Jupiter
SolarSystemParameters["JupiterRadius"]=69.911e6 #meters
SolarSystemParameters["JupiterMass"]=1.8982e27 # kg
SolarSystemParameters["JupiterOrbitRadius"]=778.57e9 # meters
SolarSystemParameters["JupiterOrbitVelocity"]=13.07e3 # meters per second

## Saturn
SolarSystemParameters["SaturnRadius"]=58.232e6 #meters
SolarSystemParameters["SaturnMass"]=5.6834e26 # kg
SolarSystemParameters["SaturnOrbitRadius"]=1.4335e12 # meters
SolarSystemParameters["SaturnOrbitVelocity"]=9.69e3 # meters per second

## Uranus
SolarSystemParameters["UranusRadius"]=25.362e6 #meters
SolarSystemParameters["UranusMass"]=8.6810e25 # kg
SolarSystemParameters["UranusOrbitRadius"]=2.8725e12 # meters
SolarSystemParameters["UranusOrbitVelocity"]=6.81e3 # meters per second

## Neptune
SolarSystemParameters["NeptuneRadius"]=24.622e6 #meters
SolarSystemParameters["NeptuneMass"]=1.02413e26 # kg
SolarSystemParameters["NeptuneOrbitRadius"]=4.4951e12 # meters
SolarSystemParameters["NeptuneOrbitVelocity"]=5.43e3 # meters per second

## Pluto
SolarSystemParameters["PlutoRadius"]=1.1883e6 #meters
SolarSystemParameters["PlutoMass"]=1.303e22 # kg
SolarSystemParameters["PlutoOrbitRadius"]=5.9064e12 # meters
SolarSystemParameters["PlutoOrbitVelocity"]=4.74e3 # meters per second

In [ ]:
## We will now be writing a full simulation
import numpy as np
import matplotlib.pyplot as plt

## Creating a force function for the gravitational force of the sun
def F(m, pos, params):
    ## Getting radius of orbit
    r = np.sqrt(pos[0]**2 + pos[1]**2)
    ## Loading in parameters from dictionary
    G = params["G"]
    m_sol = params["SunMass"] ## Sun Mass in kg
    mass_body = m ## Mass of orbiting body in kg
    ## Getting gravitational force in x direction
    F_x = -(G * m_sol * mass_body * pos[0]) / r**3
    ## Getting gravitational force in y direction
    F_y = -(G * m_sol * mass_body * pos[1]) / r**3
    ## Returning force as a numpy array
    return np.array([F_x, F_y])

## Creating a parameter dictionary for the situation
## Since the situation has changed, so do the parameters
parameters ={ #another way of writing a dictionary
'b'  :0.3,
"m"  :1.0,
"initPos" : np.array([0.0,1000.0]),
"initVel" : np.array([0.1,10.0]),
"dt" : 0.00001, ## Very small value of dt for exact solution
"g" : 9.81
        }

## Changing the step function to a Runge Kutta-4
def Step(t, pos, vel, dt):
    assert(type(pos)==np.ndarray) # checks to see if pos is an array
    ## Implementing each runge kutta order
    ## k1 is just the velocity/acceleration at the start
    k1 = vel
    k1_vel = F(t, pos, vel, parameters)/parameters["m"]
    ## k2 is the velocity/acceleration at the midpoint using k_1
    k2 = vel + (0.5 * k1_vel * dt)
    k2_vel = F(t + (dt/2), pos + (0.5 * dt * k1), vel + (0.5 * dt * k1_vel), parameters)/parameters["m"]
    ## k_3 is the velocity/acceleration at the midpoint using k_2
    k3 = vel + (0.5 * k2_vel * dt)
    k3_vel = F(t + (dt/2), pos + (0.5 * dt * k2), vel + (0.5 * dt * k2_vel), parameters)/parameters["m"]
    ## k_4 is the velocity/acceleration at the end using k_3
    k4 = vel + (k3_vel * dt)
    k4_vel = F(t + (dt), pos + (dt * k3), vel + (dt * k3_vel), parameters)/parameters["m"]
    ## Finding the final position/velocities (Using RK4)
    final_pos = pos + ((dt/6) * (k1 + 2*k2 + 2*k3 + k4))
    final_vel = vel + ((dt/6) * (k1_vel + 2*k2_vel + 2*k3_vel + k4_vel))
    ## Updating variables
    new_t = t + dt
    new_pos = np.array(final_pos)
    new_vel = np.array(final_vel)
    return (new_t, new_pos, new_vel)

## Define Orbit Function function
def Orbit(initPos, initVel, dt, parameters, mass):
    assert(type(initPos)==np.ndarray) # checks to see if pos is an array
    assert(type(initVel)==np.ndarray) # checks to see if vel is an array
    total_time = parameters["T"]
    ## Initializing lists to be returned
    times = [0]
    velocities = [initVel]
    positions = [initPos]
    ## To ensure that the simulation runs for the given time
    while times[-1] < total_time:
        t, pos, vel = Step(times[-1], positions[-1], velocities[-1], dt, mass) ## Last value in list
        times.append(t) ## Add new time
        positions.append(pos) ## Add new position
        velocities.append(vel) ## Add new velocity
    return (np.array(times), np.array(positions), np.array(velocities))


## Setting initial parameters for Earth orbiting the Sun
parameters = dict()
parameters["T"] = 365.25*24*3600 # seconds in one year
parameters["dt"] = 3600 # one hour time step
parameters["earthInitPos"] = np.array([SolarSystemParameters["EarthOrbitRadius"], 0]) # initial position in meters
parameters["earthInitVel"] = np.array([0, SolarSystemParameters["EarthOrbitVelocity"]]) # initial velocity in meters per second

## Running the orbit simulation for Earth
times, positions, velocities = Orbit(parameters["earthInitPos"], parameters["earthInitVel"],
                                     parameters["dt"], parameters,
                                     SolarSystemParameters["EarthMass"])